In [1]:
import shutil
import urllib.request as request
from contextlib import closing
import zipfile
import tarfile
import os
import glob

shapefiles = {
    "drought": "https://droughtmonitor.unl.edu/data/shapefiles_m/USDM_current_M.zip",
    
}

river_tars = {
    "rivers": "https://water.weather.gov/ahps2/shapefiles/national_shapefile_fcst_f072.tgz",
}


def clear_folder_contents(folder):
  '''
  Clears all contents of folder specified in the argument
  '''
  for file in os.listdir(folder):
    file_path = os.path.join(folder, file)
    try:
      if os.path.isfile(file_path):
        os.unlink(file_path)
      elif os.path.isdir(file_path): shutil.rmtree(file_path)
    except Exception as e:
      print(e)


def download_zip_file(file_url: str):
  '''
  Downloads a zip file and extracts it in a target directory of choice
  '''
  file = file_url.split('/')[-1]

  with closing(request.urlopen(file_url)) as r:
    with open(file, 'wb') as f:
      shutil.copyfileobj(r, f)
  
  with zipfile.ZipFile(file, "r") as zip_ref:
    zip_ref.extractall(f'data/')
    zip_ref.close()


def download_tar_file(file_url: str):
    '''
    Downloads a tar file and extracts it in a target directory of choice
    '''
    file = file_url.split('/')[-1]

    with closing(request.urlopen(file_url)) as r:
        with open(file, 'wb') as f:
          shutil.copyfileobj(r, f)
    
    tar = tarfile.open(file, mode='r')
    tar.extractall('data')
  
  
# Clear directories before getting data
clear_folder_contents('data')

# Get the shapefiles
for url in shapefiles:
    download_zip_file(shapefiles[url])
    
# Get the QPF tar files
for url in river_tars:
    download_tar_file(river_tars[url])

# Move zips
if not os.path.exists('data/zips'):
    os.makedirs('data/zips')
    
for zip_file in glob.glob(f"{os.getcwd()}/*.zip"):
  head, tail = os.path.split(zip_file)
  shutil.move(zip_file, f"data/zips/{tail}")
    
# Move tars
if not os.path.exists('data/tars'):
    os.makedirs('data/tars')
    
for tar_file in glob.glob(f"{os.getcwd()}/*.tgz"):
  head, tail = os.path.split(tar_file)
  shutil.move(tar_file, f"data/tars/{tail}")